In [72]:
   import pandas as pd

In [73]:
def prob_classA(total_distinct, train_set, column, distinct):
    info = []
    for i in range(1,len(column)-1):
        distinct_value = pd.value_counts(train_set[column[i]].values)
        start,j = 0,i-1
        arraycolumn = []
        while start<len(distinct[j]):
            get_idx = distinct[j][start]
            prob = distinct_value[get_idx]/total_distinct[0]
            arraycolumn.append(prob)
            start = start + 1
        info.append(arraycolumn)
    return info

def prob_classB(total_distinct, train_set, column, distinct):
    info = []
    for i in range(1,len(column)-1):
        distinct_value = pd.value_counts(train_set[column[i]].values)
        start,j = 0,i-1
        arraycolumn = []
        while start<len(distinct[j]):
            get_idx = distinct[j][start]
            prob = distinct_value[get_idx]/total_distinct[1]
            arraycolumn.append(prob)
            start = start + 1
        info.append(arraycolumn)
    return info   

In [74]:
def is_found(distinct_train,idx,k):
    set_DistinctIdx = distinct_train[idx]
    i,index = 0,0
    found = True
    while i < len(set_DistinctIdx) and found:
        if set_DistinctIdx[i] == k:
            index = set_DistinctIdx[i]
            found = False
        else:   
            i = i + 1
    if index == k:
        return True
    else:
        return False

def search_prob(distinct_train,idx,k):
    set_DistinctIdx = distinct_train[idx]
    i = 0
    found = True
    while i < len(set_DistinctIdx) and found:
        if set_DistinctIdx[i] != k:
            i = i + 1
        else:   
            found = False
    return i

def calculate_TestS(test_set,distinct_train,class_A,column):
    info= []
    for idx in range(1,len(column)-1):
        d = test_set.iloc[:,idx]
        prob_Arr= []
        i = idx-1
        for k,value in enumerate(d):
            if is_found(distinct_train,i,value):
                prob = search_prob(distinct_train,i,value)
                tmp = class_A[i][prob]
                
            else:
                tmp = 0
            prob_Arr.append(tmp)
        info.append(prob_Arr)
    return info

    def calculate_TestS(test_set,distinct_train,class_B,column):
        info= []
        for idx in range(1,len(column)-1):
            d = test_set.iloc[:,idx]
            prob_Arr= []
            i = idx-1
            for k,value in enumerate(d):
                if is_found(distinct_train,i,value):
                    prob = search_prob(distinct_train,i,value)
                    tmp = class_B[i][prob]
                    
                else:
                    tmp = 0
                prob_Arr.append(tmp)
            info.append(prob_Arr)
        return info


In [75]:

def mul_classA(test_set,resultA,prob_disA):
    class_A = []
    for i in range(len(test_set)):
        result = prob_disA
        for j in range(len(resultA)):
            result = resultA[j][i] * result
        class_A.append(result)
    return class_A

def mul_classB(test_set,resultB,prob_disB):
    class_B = []
    for i in range(len(test_set)):
        result = prob_disB
        for j in range(len(resultB)):
            result = resultB[j][i] * result
        class_B.append(result)
    return class_B


In [76]:
def get_class(a,b,distinct,total_class,test_set,idx):
    precision, Recall,TN ,TP, FN, FP, d = 0,0,0,0,0,0,0
    idx_v = idx - len(test_set)
    for i in range(len(a)):
        if a[i]>b[i]:
            d = test_set.iloc[:,15]
            if d[idx_v]==(distinct[14][0]):
                TN = TN + 1
            else:
                FN = FN + 1
        else:
            d = test_set.iloc[:,15]
            if d[idx_v]==(distinct[14][1]):
                TP = TP + 1
            else:
                FP = FP + 1
        idx_v = idx_v + 1
    print("True Positive :",TP," False Positive: ",FP," True Negative: ",TN," False Negative: ",FN)
    accuracy = (TP/len(a) * 100)
    if (TP+FP) == 0 :
        precision = 0
    elif (TP+FN)==0:
        Recall = 0
    else:
        precision = (TP/(TP+FP)*100)
        Recall = (TP/(TP+FN)*100)
    print("precision: ",round(precision,2),"% Recall: ",round(Recall,2),"%")
    print("The accuracy is ", round(accuracy,2),'%')


In [78]:
def testData(idxVal,df,idx,prev_idx):
    if idxVal == 1:
        test_set = df[:idx]
    else:
        if idxVal != 8:
            test_set = df[prev_idx:idx]
        else:
            test_set = df[idx:]
    return test_set

def trainData(idxVal,df,idx,prev_idx):
    
    if idxVal == 1:
        train_set = df[idx:]
    else:
        if idxVal != 8:
            train_set = pd.concat([df[:prev_idx],df[idx:]])
        else:
            train_set = df[:idx] 
    return train_set

In [81]:
def main():

 
    df = pd.read_csv('csv_result-adult1.csv')
    prev_idx = 0
    testset = int(len(df)/7)
    for i in range(1,8):
        idx = testset * i
        train_set = trainData(i,df,idx,prev_idx)
        test_set = testData(i,df,idx,prev_idx)

        print("============== ",i," FOLD ===================")

        column = [col for col in df.columns]

        distinct_train = [train_set[column[idx]].unique() for idx in range(1,len(column))]
        distinct_test = [test_set[column[idx]].unique() for idx in range(1,len(column))]

        total_distinct = pd.value_counts(train_set['class'].values)
        total_class = pd.value_counts(test_set['class'].values)

        prob_disA = total_distinct[0]/len(train_set)
        prob_disB = total_distinct[1]/len(train_set)

        class_A = prob_classA(total_distinct,train_set,column,distinct_train)
        class_B = prob_classB(total_distinct,train_set,column,distinct_train)

        resultA = calculate_TestS(test_set,distinct_train,class_A,column)
        resultB = calculate_TestS(test_set,distinct_train,class_B,column)

        a = mul_classA(test_set,resultA,prob_disA)
        b = mul_classB(test_set,resultB,prob_disB)

        get_class(a,b,distinct_train,total_class,test_set,idx)
        prev_idx = idx


In [82]:
main()

==============  1  FOLD ===================
True Positive : 2875  False Positive:  936  True Negative:  0  False Negative:  0
precision:  75.44 % Recall:  100.0 %
The accuracy is  75.44 %
==============  2  FOLD ===================
True Positive : 926  False Positive:  2885  True Negative:  0  False Negative:  0
precision:  24.3 % Recall:  100.0 %
The accuracy is  24.3 %
==============  3  FOLD ===================
True Positive : 955  False Positive:  2856  True Negative:  0  False Negative:  0
precision:  25.06 % Recall:  100.0 %
The accuracy is  25.06 %
==============  4  FOLD ===================
True Positive : 941  False Positive:  2870  True Negative:  0  False Negative:  0
precision:  24.69 % Recall:  100.0 %
The accuracy is  24.69 %
==============  5  FOLD ===================
True Positive : 958  False Positive:  2853  True Negative:  0  False Negative:  0
precision:  25.14 % Recall:  100.0 %
The accuracy is  25.14 %
==============  6  FOLD ===================
True Positive : 91